In [1]:
!pip install datasets transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 432 kB 4.9 MB/s 
     |████████████████████████████████| 4.9 MB 58.0 MB/s 
     |████████████████████████████████| 115 kB 75.6 MB/s 
     |████████████████████████████████| 212 kB 69.4 MB/s 
     |████████████████████████████████| 163 kB 67.6 MB/s 
     |████████████████████████████████| 127 kB 48.6 MB/s 
     |████████████████████████████████| 6.6 MB 39.3 MB/s 
     |████████████████████████████████| 1.3 MB 50.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [1]:
import os
import os.path

import pandas as pd
import cv2
from PIL import Image
import torch.utils.data as data


import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence

import numpy as np
import cv2
import os
import json
from matplotlib import pyplot as plt

import time
from tqdm.notebook import tqdm_notebook as tnb
from tqdm.auto import tqdm  # for notebooks

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
config_json = {
    "batch_size": 512,
    "num_workers": 1,
    "save_dir": "/content/drive/MyDrive/ML/MoreTech/ModelA",
    "checkpoint_path": "/content/drive/MyDrive/ML/MoreTech/ModelA/model_last.ckpt",
    "log_path":{
        "train": "/content/drive/MyDrive/ML/MoreTech/ModelA/trian_score.log",
        "val": "/content/drive/MyDrive/ML/MoreTech/ModelA/val_score.log"
    },
    "root": "/content/CompCars/data/image",
    "annotations":{
        "train": "/content/CompCars/annotation/train.txt",
        "val": "/content/CompCars/annotation/val.txt"
    },

}

**Data preparation**

In [54]:
from transformers import BertForMaskedLM,BertTokenizer, pipeline
class BertEmbeddings:
    def __init__(self, DEVICE="cpu"):
        self.model=BertForMaskedLM.from_pretrained('sberbank-ai/ruBert-base')
        self.model.cls.predictions.decoder = nn.Identity()
        self.model = self.model.to(DEVICE)
        self.device = DEVICE
        self.tokenizer=BertTokenizer.from_pretrained('sberbank-ai/ruBert-base', do_lower_case=False)
    def __call__(self, text):
        input_ids = self.tokenizer(
            [text],
            truncation=True,
            return_tensors="pt",
        )["input_ids"].to(self.device)
        #print(self.model(input_ids).logits.shape)
        out = self.model(input_ids).logits
        out2=torch.mean(out[0], dim=0)
        print(out2.shape)
        return out2

from transformers import MBartTokenizer, MBartForConditionalGeneration
class SummarizationPipeline:
    def __init__(self, DEVICE="cpu"):
        model_name = "IlyaGusev/mbart_ru_sum_gazeta"
        self.tokenizer = MBartTokenizer.from_pretrained(model_name)
        self.model = MBartForConditionalGeneration.from_pretrained(model_name)
        self.model = self.model.to(DEVICE)
        self.device = DEVICE
    def __call__(self, text, max_len = 600):
        text = str(text)
        input_ids = self.tokenizer(
            [text],
            max_length = max_len,
            truncation=True,
            return_tensors="pt",
        )["input_ids"].to(self.device)

        output_ids = self.model.generate(
            input_ids=input_ids,
            no_repeat_ngram_size=4
        )[0]
        summary = self.tokenizer.decode(output_ids, skip_special_tokens=True)
        return summary

**DATALOADER**

In [18]:
def catToTensor(cat):
    if cat == 1:
        return torch.tensor([1, 0, 0])
    if cat == 2:
        return torch.tensor([0, 1, 0])
    else:
        return torch.tensor([0, 0, 1])
def prepareData(dataPath, dataOut):
    df = pd.read_csv(dataPath, index_col=0)
    tqdm.pandas()
    #summarize = SummarizationPipeline(DEVICE)
    #df['summary'] = df.progress_apply(lambda row: summarize(row['text'], max_len=1000), axis = 1)
    #df.to_csv(dataOut)
    embedize = BertEmbeddings(DEVICE)
    for i, summary in enumerate(df["summary"]):
      torch.save(embedize(summary).cpu().detach(), open(f"tensors/torch{i}.trch", "wb"))
    # df['tensor'] = df.progress_apply(lambda row: embedize(row['summary']).cpu().detach().numpy().tolist(), axis = 1)
    #df.to_csv(dataOut)
    df['target'] = df.progress_apply(lambda row: catToTensor(int(row['category'])).cpu().detach().numpy().tolist(), axis = 1)
    
    df.to_csv(dataOut)

In [6]:
a = pd.DataFrame(pd.read_csv("/content/data.csv", index_col=0).values, columns=["link", "text", "title", "category", "summary", "tensor", "target"])
a.to_csv("/content/data2.csv")

In [55]:

prepareData('/content/data2.csv', '/content/data.csv')

Some weights of the model checkpoint at sberbank-ai/ruBert-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size

  0%|          | 0/575 [00:00<?, ?it/s]

In [69]:
b = pd.read_csv("/content/data.csv")

In [76]:
b['target'].tolist()[0]

'[1, 0, 0]'

In [10]:
i=0
torch.tensor(torch.load(f"/content/tensors/torch{i}.trch").cpu().detach().numpy())

tensor([-3.5136,  1.3936,  2.7599,  ..., -0.2965, -0.1117, -0.1402])

In [56]:
torch.load(f"/content/tensors/torch{2}.trch").shape

torch.Size([768])

In [62]:
class CSVDataset(data.Dataset):
    def __init__(self, root, is_inference=False):
        self.root = root
        #self.embs = pd.read_csv(root)['tensor'].tolist()
        self.targets = pd.read_csv(root)['target'].tolist()
        self.embs=[]
        for i in range(0, len(self.targets)):
          self.embs.append(torch.load(f"/content/tensors/torch{i}.trch"))
        self.is_inference = is_inference

    def __getitem__(self, index):
        
        if self.is_inference:
            #embs = torch.tensor(torch.load(f"/content/tensors/torch{index}.trch"))
            embs=self.embs[index]
        else:
            #embs = torch.tensor(torch.load(f"/content/tensors/torch{index}.trch"))
            embs=self.embs[index]
            target = self.targets[index]
        
        target = torch.tensor(eval(target)).double()
        if self.is_inference:
            return embs
        else:
            
            return embs, target

    def __len__(self):
        return len(self.targets)

In [27]:
def get_dataloaders(config):
    print("Preparing train reader...")
    train_dataset = CSVDataset(root="/content/data.csv")
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=config['batch_size'],
        shuffle=True,
        num_workers=config['num_workers'],
        pin_memory=True,
        drop_last=True
    )

    return train_loader

MODEL AND OTHER STUFF

In [28]:
class AverageMeter:
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [80]:
class Classifier(nn.Module):
    def __init__(
            self, input_size=768, number_classes=3,
    ):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_size, 2000),
            nn.GELU(),
            nn.Linear(2000, 1000),
            nn.GELU(),
            nn.Linear(1000, number_classes),
            #nn.Softmax(),
        )
    def forward(self, x):
        x = self.classifier(x)
        return x

In [101]:
def train_loop(model, criterion, optimizer, train_loader, n):
    loss_avg = AverageMeter()
    acc_stat = AverageMeter()
    start_time = time.time()
    #for embeddings, targets in tnb(train_loader, desc="Training cycle: "):
    for embeddings, targets in train_loader:
        model.train()
        model.zero_grad()
        embeddings = embeddings.to(DEVICE)
        targets = targets.to(DEVICE)
        output = model(embeddings)

        
        loss = criterion(output, targets)
        loss_avg.update(loss.item(), 1)
        

        output2 = output.softmax(dim=1)
        output2 = output2.cpu().detach().numpy()
        predict = np.argmax(output2, axis=1)

        gt = targets.cpu().detach().numpy()
        targ = np.argmax(gt, axis=1)
        acc = np.mean(predict == targ)

        acc_stat.update(acc, 1)
        loss.backward()
        optimizer.step()
    for param_group in optimizer.param_groups:
        lr = param_group['lr']
    print(f"{n} epoch, Loss: {loss_avg.avg:.5f}, acc: {acc_stat.avg:.4f}, LR: {lr:.7f}, Time for cycle: {(time.time() - start_time):.2f} sec")
    return loss_avg.avg

def val_loop(model, criterion, data_loader):
    start_time = time.time()
    loss_avg = AverageMeter()
    acc_stat = AverageMeter()
    for embs, targets in tnb(data_loader, desc='Validation: '):
        model.eval()
        embs = embs.to(DEVICE)
        targets = targets.to(DEVICE)
        with torch.no_grad():
            output = model(embs)
        predict = np.argmax(output.detach(), axis=0)
        gt = targets.detach().cpu().numpy()
        acc = np.mean(gt == predict)
        acc_stat.update(acc, 1)
        loss = criterion(output, targets)

        loss_avg.update(loss.item(), 1)
    print(f'\nValidation, loss: {loss_avg.avg:.4f}, acc: {acc_stat.avg:.4f}, Time taken: {(time.time() - start_time):.2f} sec')
    return loss_avg.avg

def model_load(config, preTrained=False, checkpoint_path=""):
    model = Classifier()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.1,
                                  weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.1, steps_per_epoch=10, epochs=10)
    if preTrained:
        """Loading model"""
        checkpoint = torch.load(checkpoint_path, map_location=torch.device('cuda') )
        model.load_state_dict(     checkpoint['model'])
        model=model.to(DEVICE)
        optimizer.load_state_dict( checkpoint['optimizer'])
        scheduler.load_state_dict( checkpoint['scheduler'])
        epoch =                    checkpoint['epoch']
        score =                    checkpoint['score']
        return model, optimizer, criterion, scheduler, epoch, score
    else:
        return model.to(DEVICE), optimizer, criterion, scheduler, 0, 0
def save_model(model, optimizer, scheduler, epoch, score, path):
    checkpoint={
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'scheduler': scheduler.state_dict(),
        'epoch': epoch,
        'score': score
    }
    torch.save(checkpoint, path)
def Train_log(score, path):
    arr=np.array([score])
    if os.path.exists(path):
        with open(path, 'rb') as f:
            arr = np.load(f)
        arr = np.append(arr, score)
    with open(path, 'wb') as f:
        np.save(f, arr)

def train(config):
    train_loader = get_dataloaders(config)
    model, optimizer, criterion, scheduler, epoch, score = model_load(config, False, config["checkpoint_path"])
    lambda1 = lambda epoch: 0.7 ** (epoch + 2)
    #scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)
    print("Model loaded")
    criterion = criterion.to(DEVICE)
    model=model.to(DEVICE)
    for e in tnb(range(epoch, 20)):
        score = train_loop(model, criterion, optimizer, train_loader, e)
        scheduler.step(score)
        save_model(model, optimizer, scheduler, e, score, config["checkpoint_path"])
        Train_log(score, config["log_path"]["train"])
        print("Data saved \n")

In [103]:
#DEVICE='cpu'
train(config_json)

Preparing train reader...
Model loaded


  0%|          | 0/20 [00:00<?, ?it/s]

0 epoch, Loss: 1.09666, acc: 0.3633, LR: 0.0040000, Time for cycle: 0.15 sec
Data saved 

1 epoch, Loss: 15.32469, acc: 0.6719, LR: 0.0040000, Time for cycle: 0.14 sec
Data saved 

2 epoch, Loss: 6.93581, acc: 0.2969, LR: 0.0040000, Time for cycle: 0.14 sec
Data saved 

3 epoch, Loss: 16.63776, acc: 0.1250, LR: 0.0040000, Time for cycle: 0.16 sec
Data saved 

4 epoch, Loss: 1.08595, acc: 0.5039, LR: 0.0040000, Time for cycle: 0.17 sec
Data saved 

5 epoch, Loss: 0.81047, acc: 0.7793, LR: 0.0040000, Time for cycle: 0.15 sec
Data saved 

6 epoch, Loss: 0.69119, acc: 0.7988, LR: 0.0040000, Time for cycle: 0.16 sec
Data saved 

7 epoch, Loss: 0.54224, acc: 0.8242, LR: 0.0040000, Time for cycle: 0.16 sec
Data saved 

8 epoch, Loss: 0.37843, acc: 0.8613, LR: 0.0040000, Time for cycle: 0.18 sec
Data saved 

9 epoch, Loss: 0.39919, acc: 0.8770, LR: 0.0040000, Time for cycle: 0.16 sec
Data saved 

10 epoch, Loss: 0.39449, acc: 0.8906, LR: 0.0040000, Time for cycle: 0.16 sec
Data saved 

11 epoc